In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Data from CSV files 

In [83]:
Traindf=pd.read_csv('../input/titanic/train.csv')
Testdf=pd.read_csv('../input/titanic/test.csv')

Actual Survived

In [84]:
gender = pd.read_csv('../input/titanic/gender_submission.csv')
gender.head()
actual_survival_test_data = gender['Survived'].values

In [85]:
Traindf.head()

In [86]:
Traindf.shape

We have 891 of total training data ,each contains 12 columns including label.

In [87]:
sum(Traindf['Survived'].isnull())

In [88]:
a=Traindf['Survived'].value_counts()
b=Traindf['Survived'].value_counts().keys()

In [89]:
import seaborn as sns

**Gender based distribution**

In [90]:
sns.countplot(x='Sex',data=Traindf)

**How many of them survived?**

In [91]:
sns.countplot(x='Survived',linewidth=6,data=Traindf)

**Classes distribution with respect to age**

In [92]:
sns.boxplot(x='Pclass',y='Age',data=Traindf)
plt.xlabel('Passenger Class')
plt.ylabel('Age')
plt.title('Classes distribution with respect to age')

**Classes distribution with respect to age**

In [93]:
sns.barplot(x = 'Pclass', y  = 'Survived', data = Traindf)
plt.xlabel('Passenger Class')
plt.ylabel('Survival Rate')
plt.title('Survival Rates by Passenger Class on the Titanic')

**Unique Values**

In [94]:
Traindf.nunique().sort_values(ascending = False).to_frame().T

In [95]:
Traindf.head(50)

# PreProcessing


In [96]:
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [97]:
Traindf['Age'].fillna((Traindf['Age'].mean()), inplace=True)
Testdf['Age'].fillna((Testdf['Age'].mean()), inplace=True)

In [98]:
Features = Traindf[['Pclass', 'Sex', 'SibSp', 'Parch']]
X = pd.get_dummies(Features)
X = preprocessing.StandardScaler().fit(X).transform(X)

In [99]:
y = Traindf['Survived'].values

# Logistic Regression


# Model Training


In [100]:
from sklearn.linear_model import LogisticRegression
lreg =LogisticRegression(max_iter=4000,penalty='l2')
param={'C':[0.1,0.001,0.01,0.05],'solver':['lbfgs','newton-cg', 'saga','sag']}
Gridcv=GridSearchCV(lreg,param,cv=6)
Gridcv.fit(X,y)



In [101]:
print("tuned hpyerparameters :(best parameters) ",Gridcv.best_params_)
print("accuracy :",Gridcv.best_score_)

# Model Testing

In [102]:
Features1 = Testdf[['Pclass', 'Sex', 'SibSp', 'Parch', ]]
X1 = pd.get_dummies(Features1)
X1 = preprocessing.StandardScaler().fit(X1).transform(X1)

In [103]:
logreg_prediction = Gridcv.predict(X1)
logreg_accuracy = metrics.accuracy_score(actual_survival_test_data, logreg_prediction)
print("The logistic regression's accuracy was: ", metrics.accuracy_score(actual_survival_test_data, logreg_prediction))

# Decision Tree
# Model Training

In [104]:
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

tree = DecisionTreeClassifier()
tree_cv = GridSearchCV(tree, parameters, cv = 10)
tree_cv.fit(X, y)
print("tuned hpyerparameters :(best parameters) ",tree_cv.best_params_)
print("accuracy :",tree_cv.best_score_)

# Model Testing

In [105]:
tree_prediction = tree_cv.predict(X1)
tree_accuracy = metrics.accuracy_score(actual_survival_test_data, tree_prediction)
tree_accuracy


# Random Forest
# Model Training

In [106]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=130, max_depth=3, random_state=1)
model.fit(X, y)
predictions = model.predict(X1)

In [107]:
rf_prediction = model.predict(X1)
rf_accuracy = metrics.accuracy_score(actual_survival_test_data,rf_prediction)
rf_accuracy

In [111]:
output = pd.DataFrame({'PassengerId': Testdf.PassengerId, 'Survived': rf_prediction})
output.to_csv('submission.csv', index=False)